# **Imports**

In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split, cross_val_score
from statistics import mean
from nltk.corpus import wordnet 
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from itertools import combinations
from time import time
from collections import Counter
import operator
from xgboost import XGBClassifier
import math
from sklearn.linear_model import LogisticRegression
from synonyms import syno
from synonyms import generatesymptoms2
import joblib

warnings.simplefilter("ignore")

In [ ]:
from LR import LR
from LR import diseaseDetail
from randomrf import RandF
from knneigbh import knn
from decisionT import decisionT
from multiLP import mlp
from IRsvm import SVM
from mnb import MNB
from IRadaboost import ADA
from IRgdb import GDB
from IRxgb import XGB

In [ ]:
import nltk
nltk.download('all')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Data Preprocess and Score Calculation**

In [ ]:
df_comb = pd.read_csv("/content/drive/MyDrive/combinational (3).csv") # Disease combination
df_norm = pd.read_csv("/content/drive/MyDrive/normal (3).csv") # Individual Disease

X = df_comb.iloc[:, 1:]
Y = df_comb.iloc[:, 0:1]

In [ ]:
#path of the saved models
lr_filename = '/content/drive/MyDrive/lr_model.sav'
rf_filename = '/content/drive/MyDrive/rf_model.sav'
knn_filename = '/content/drive/MyDrive/knn_model.sav'
dt_filename = '/content/drive/MyDrive/dt_model.sav'
mlp_filename = '/content/drive/MyDrive/mlp_model.sav'
svm_filename = '/content/drive/MyDrive/svm_model.sav'
mnb_filename = '/content/drive/MyDrive/mnb_model.sav'
ada_filename = '/content/drive/MyDrive/ada_model.sav'
gdb_filename = '/content/drive/MyDrive/gdb_model.sav'
xgb_filename = '/content/drive/MyDrive/xgb_model.sav'

In [ ]:
lr = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr = lr.fit(X, Y)
# save the model to disk
filename = '/content/drive/MyDrive/lr_model.sav'
joblib.dump(lr, filename)

['/content/drive/MyDrive/lr_model.sav']

In [ ]:
lr = joblib.load(lr_filename)
scoresLR = cross_val_score(lr, X, Y, cv=5)

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rf
rfmod=rf(n_estimators=20,criterion='entropy')
rfmod.fit(X,Y)
filename = '/content/drive/MyDrive/rf_model.sav'
joblib.dump(rfmod, filename)

['/content/drive/MyDrive/rf_model.sav']

In [ ]:
rfmod = joblib.load(rf_filename)
scoresRF = cross_val_score(rfmod, X, Y, cv=5)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knn
knn_obj=knn()
knn_obj.fit(X,Y)
filename = '/content/drive/MyDrive/knn_model.sav'
joblib.dump(knn_obj, filename)

['/content/drive/MyDrive/knn_model.sav']

In [ ]:
knn_obj = joblib.load(knn_filename)
scoresknn = cross_val_score(knn_obj, X, Y, cv=5)

In [ ]:
from sklearn.tree import DecisionTreeClassifier as dtree
dtree_mod=dtree(criterion='entropy')
dtree_mod.fit(X,Y)
filename = '/content/drive/MyDrive/dt_model.sav'
joblib.dump(dtree_mod, filename)

['/content/drive/MyDrive/dt_model.sav']

In [ ]:
dtree_mod = joblib.load(dt_filename)
scoresDT = cross_val_score(dtree_mod, X, Y, cv=5)

In [ ]:
from sklearn.neural_network import MLPClassifier as mlp
obj=mlp(hidden_layer_sizes=(200,50),max_iter=1000,solver='adam')
obj.fit(X,Y)
filename = '/content/drive/MyDrive/mlp_model.sav'
joblib.dump(obj, filename)

['/content/drive/MyDrive/mlp_model.sav']

In [ ]:
obj = joblib.load(mlp_filename)
scoresmlp = cross_val_score(obj, X, Y, cv=5)

In [ ]:
from sklearn import svm
#svmclf = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)
svmclf = svm.SVC(kernel='linear', probability=True)
#svmclf = svm.LinearSVC()
svmclf.fit(X,Y)
filename = '/content/drive/MyDrive/svm_model.sav'
joblib.dump(svmclf, filename)

['/content/drive/MyDrive/svm_model.sav']

In [ ]:
svmclf= joblib.load(svm_filename)
scoressvm = cross_val_score(svmclf, X, Y, cv=5)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNBclf = MultinomialNB(alpha=1.0, fit_prior=True)
MNBclf.fit(X, Y)
filename = '/content/drive/MyDrive/mnb_model.sav'
joblib.dump(MNBclf, filename)

['/content/drive/MyDrive/mnb_model.sav']

In [ ]:
MNBclf= joblib.load(mnb_filename)
scoresmnb = cross_val_score(MNBclf, X, Y, cv=5)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=50, learning_rate=1)
adb_obj = abc.fit(X,Y)
filename = '/content/drive/MyDrive/ada_model.sav'
joblib.dump(adb_obj, filename)

['/content/drive/MyDrive/ada_model.sav']

In [ ]:
adb_obj= joblib.load(ada_filename)
scoresabc = cross_val_score(adb_obj, X, Y, cv=5)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gdb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50)
gdb.fit(X, Y)
filename = '/content/drive/MyDrive/gdb_model.sav'
joblib.dump(gdb, filename)

['/content/drive/MyDrive/gdb_model.sav']

In [ ]:
gdb= joblib.load(gdb_filename)
scoresgdb = cross_val_score(gdb, X, Y, cv=5)

In [ ]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="multi:softprob")
xgb_model.fit(X, Y)
filename = '/content/drive/MyDrive/xgb_model.sav'
joblib.dump(xgb_model, filename)

['/content/drive/MyDrive/xgb_model.sav']

In [ ]:
xgb_model= joblib.load(xgb_filename)
scoresxgb = cross_val_score(xgb_model, X, Y, cv=5)

In [ ]:
X = df_norm.iloc[:, 1:]
Y = df_norm.iloc[:, 0:1]

In [ ]:
dataset_symptoms = list(X.columns)

# **Input From User**

In [ ]:
user_symptoms1 = str(input("Please enter symptoms separated by comma(,):\n")).lower().split(',')
user_symptoms = generatesymptoms2(user_symptoms1)

Please enter symptoms separated by comma(,):
fever
After query expansion done by using the symptoms entered
['febricity fever pyrexia febrility feverishness']


In [ ]:
found_symptoms = set()
for idx, data_sym in enumerate(dataset_symptoms):
    data_sym_split=data_sym.split()
    for user_sym in user_symptoms:
        count=0
        for symp in data_sym_split:
            if symp in user_sym.split():
                count+=1
        if count/len(data_sym_split)>0.5:
            found_symptoms.add(data_sym)
found_symptoms = list(found_symptoms)

In [ ]:
print("Top matching symptoms from your search!")
for idx, symp in enumerate(found_symptoms):
    print(idx,":",symp)
    
select_list = input("\nPlease select the relevant symptoms. Enter indices (separated-space):\n").split()

dis_list = set()
final_symp = [] 
counter_list = []
for idx in select_list:
    symp=found_symptoms[int(idx)]
    final_symp.append(symp)
    dis_list.update(set(df_norm[df_norm[symp]==1]['label_dis']))
   
for dis in dis_list:
    row = df_norm.loc[df_norm['label_dis'] == dis].values.tolist()
    row[0].pop(0)
    for idx,val in enumerate(row[0]):
        if val!=0 and dataset_symptoms[idx] not in final_symp:
            counter_list.append(dataset_symptoms[idx])

Top matching symptoms from your search!
0 : fever

Please select the relevant symptoms. Enter indices (separated-space):
0


In [ ]:
dict_symp = dict(Counter(counter_list))
dict_symp_tup = sorted(dict_symp.items(), key=operator.itemgetter(1),reverse=True)   
#print(dict_symp_tup) 

In [ ]:
 
found_symptoms=[]
count=0
for tup in dict_symp_tup:
    count+=1
    found_symptoms.append(tup[0])
    if count%5==0 or count==len(dict_symp_tup):
        print("\nCommon co-occuring symptoms:")
        for idx,ele in enumerate(found_symptoms):
            print(idx,":",ele)
        select_list = input("Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:\n").lower().split();
        if select_list[0]=='no':
            break
        if select_list[0]=='-1':
            found_symptoms = [] 
            continue
        for idx in select_list:
            final_symp.append(found_symptoms[int(idx)])
        found_symptoms = [] 


Common co-occuring symptoms:
0 : headache
1 : cough
2 : vomiting
3 : rash
4 : muscle pain
Do you have have of these symptoms? If Yes, enter the indices (space-separated), 'no' to stop, '-1' to skip:
no


In [ ]:
print("\nFinal list of Symptoms that will be used for prediction:")
sample_x = [0 for x in range(0,len(dataset_symptoms))]
for val in final_symp:
    print(val)
    sample_x[dataset_symptoms.index(val)]=1


Final list of Symptoms that will be used for prediction:
fever


# **Logistic Regression**

In [ ]:
print("Logistic Regression Accuracy : ",np.mean(scoresLR)*100)

Logistic Regression Accuracy :  89.88417348843889


In [ ]:
LR(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresLR)


Top 10 diseases predicted based on symptoms by Logistic Regression
0 Disease name: Epidemic Relapsing fever 	Probability: 89.88%
1 Disease name: Hand, Foot and Mouth Disease 	Probability: 89.88%
2 Disease name: Chikungunya fever 	Probability: 89.88%
3 Disease name: Pericarditis 	Probability: 89.88%
4 Disease name: Whitmore's disease 	Probability: 89.88%
5 Disease name: Necrotizing fasciitis 	Probability: 89.88%
6 Disease name: Otitis Media 	Probability: 89.88%
7 Disease name: Puerperal sepsis 	Probability: 89.88%
8 Disease name: Montezuma's Revenge 	Probability: 89.88%
9 Disease name: Tularemia 	Probability: 89.88%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
-1


# **Random Forest**

In [ ]:
print("Random Forest Accuracy : ",np.mean(scoresRF)*100)

Random Forest Accuracy :  89.9789063622002


In [ ]:
RandF(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresRF)


Top 10 diseases predicted based on symptoms by Random Forest
0 Disease name: Epidemic Relapsing fever 	Probability: 86.93%
1 Disease name: Chikungunya fever 	Probability: 86.93%
2 Disease name: Tularemia 	Probability: 86.93%
3 Disease name: Scrub typhus 	Probability: 86.93%
4 Disease name: Rheumatic fever and rheumatic heart disease 	Probability: 86.93%
5 Disease name: Necrotizing fasciitis 	Probability: 86.93%
6 Disease name: Sinusitis 	Probability: 86.93%
7 Disease name: Erysipelis 	Probability: 86.93%
8 Disease name: Isosporiasis 	Probability: 43.46%
9 Disease name: Erythema nodosum 	Probability: 43.46%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
-1


# **KNN**

In [ ]:
knn1(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresknn)

In [ ]:
print("KNN Accuracy : ",np.mean(scoresknn)*100)

KNN Accuracy :  89.9789063622002


# **Decision Tree**

In [ ]:
print("Decision Tree Accuracy : ",np.mean(scoresDT)*100)

Decision Tree Accuracy :  72.62798003734024


In [ ]:
decisionT(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresDT)


Top 10 diseases predicted based on symptoms by Decision Tree
0 Disease name: Epidemic Relapsing fever 	Probability: 72.57%
1 Disease name: Epiglottitis 	Probability: 72.57%
2 Disease name: Erysipelis 	Probability: 72.57%
3 Disease name: lactose intolerance 	Probability: 36.29%
4 Disease name: Epilepsy 	Probability: 36.29%
5 Disease name: Endemic syphilis 	Probability: 36.29%
6 Disease name: Endometriosis 	Probability: 36.29%
7 Disease name: Endothrix 	Probability: 36.29%
8 Disease name: Enterobiasis 	Probability: 36.29%
9 Disease name: Zoster 	Probability: 36.29%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
-1


# **Multilayer Perceptron**

In [ ]:
mlp1(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresmlp)

In [ ]:
print("MLP Accuracy : ",np.mean(scoresmlp)*100)

MLP Accuracy :  89.42989731437599


# **SVM**

In [ ]:
print("SVM Accuracy : ",np.mean(scoressvm)*100)

SVM Accuracy :  87.9904136148212


In [ ]:
SVM(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoressvm)


Top 10 diseases predicted based on symptoms by SVM
0 Disease name: Polycystic ovary syndrome (PCOS) 	Probability: 44.0%
1 Disease name: Lead poisoning 	Probability: 44.0%
2 Disease name: Postanginal septicemia 	Probability: 44.0%
3 Disease name: Heat-Related Illnesses and Heat waves 	Probability: 44.0%
4 Disease name: Venezuelan Equine encephalitis 	Probability: 44.0%
5 Disease name: Anthrax 	Probability: 44.0%
6 Disease name: Carpal Tunnel Syndrome 	Probability: 44.0%
7 Disease name: Brain Tumour 	Probability: 44.0%
8 Disease name: Omsk hemorrhagic fever 	Probability: 44.0%
9 Disease name: Disseminated Intravascular Coagulation 	Probability: 44.0%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
-1


# **M Naive Bayes**

In [ ]:
print("Naive Bayes Accuracy : ",np.mean(scoresmnb)*100)

Naive Bayes Accuracy :  81.3016479965532


In [ ]:
MNB(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresmnb)


Top 10 diseases predicted based on symptoms by Random Forest
0 Disease name: Epidemic Relapsing fever 	Probability: 81.3%
1 Disease name: Hand, Foot and Mouth Disease 	Probability: 81.3%
2 Disease name: Chikungunya fever 	Probability: 81.3%
3 Disease name: Middle East respiratory syndrome coronavirus (MERS‐CoV) 	Probability: 81.3%
4 Disease name: Strep. throat 	Probability: 81.3%
5 Disease name: Tetanus 	Probability: 81.3%
6 Disease name: Listerosis 	Probability: 81.3%
7 Disease name: Shigellosis 	Probability: 81.3%
8 Disease name: Lassa hemorrhagic fever 	Probability: 81.3%
9 Disease name: Chagas disease 	Probability: 81.3%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
-1


# **Ada Boost**

In [ ]:
print("Ada Boost Accuracy : ",np.mean(scoresabc)*100)

In [ ]:
ADA(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresabc)

# **Gradient Boosting**

In [ ]:
print("Gradient Boosting Accuracy : ",np.mean(scoresgdb)*100)

Gradient Boosting Accuracy :  80.94138661496481


In [ ]:
GDB(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresgdb)


Top 10 diseases predicted based on symptoms by Gradient Booster
0 Disease name: Epidemic Relapsing fever 	Probability: 82.29%
1 Disease name: Typhoid fever 	Probability: 82.29%
2 Disease name: German measles 	Probability: 82.29%
3 Disease name: Chikungunya fever 	Probability: 82.29%
4 Disease name: Food Poisoning 	Probability: 82.29%
5 Disease name: Strep. throat 	Probability: 82.29%
6 Disease name: Toxic Shock Syndrome 	Probability: 82.29%
7 Disease name: Amebic meningoencephalitis 	Probability: 82.29%
8 Disease name: Tularemia 	Probability: 82.29%
9 Disease name: Winter diarrhea 	Probability: 41.14%

More details about the disease? Enter index of disease or '-1' to discontinue and close the system:
-1


# **Extreme Gradient Boosting Accuracy**

In [ ]:
print("Extreme Gradient Boosting Accuracy : ",np.mean(scoresxgb)*100)

Extreme Gradient Boosting Accuracy :  78.83783929340801


In [ ]:
XGB(X,Y,sample_x,df_norm,dataset_symptoms,final_symp,scoresxgb)